### Data Loading

In [ ]:
import json
import pydeck
import pickle
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
import pandas as pd
import os
import shapefile
import time

In [ ]:
data_dir = "./data"

In [ ]:
# load location index
with open(os.path.join(data_dir, "akl_loc_idx.pkl"), 'rb') as f:
    loc_idx = pickle.load(f) # datazone to point index
    idx_loc = {v:k for k, v in loc_idx.items()} # point index to datazone    
    print(f" -- loaded location index with dimension {len(loc_idx)}")

# load time index
with open(os.path.join(data_dir, "akl_t_idx.pkl"), 'rb') as f:
    t_idx = pickle.load(f) # datetime to time index
    idx_t = {v:k for k, v in t_idx.items()} # time index to datetime
    print(f" -- loaded time index with dimension {len(t_idx)}")
    
# load precomputed odt
with open(os.path.join(data_dir, "akl_odt.npy"), 'rb') as f:    
    odt = np.load(f)
    print(f" -- loaded odt cube with dimensions {odt.shape}")

In [ ]:
# show odt time rane
times = list(t_idx.keys())
print(min(times), "-", max(times))

In [ ]:
# load polygon data
with open(os.path.join(data_dir, "akl_polygons_id.geojson")) as f:
    polys = json.load(f)

In [ ]:
# load shapefile of points (data zone population centeroids)
sf_path = os.path.join(data_dir, "akl_points.shp")
sf = shapefile.Reader(sf_path)
records = sf.records()
coords = {}
for i, r in enumerate(records):    
    coords[r[0]] = sf.shape(i).points[0]
sf.close()

In [ ]:
# load IMD
imd = pd.read_csv(os.path.join(data_dir, "akl_imd.csv"), index_col="DZ2018")
imd.head()

In [ ]:
# load vdr

vdr = pd.read_csv(os.path.join(data_dir, "vdr_values.csv"), index_col="lzuid").dropna()
vdr.index = vdr.index.astype(np.int32)
#vdr = pd.read_csv("../data/vdr_values.csv").dropna()

# replace 'S' suppressed values with 0
# vdr["count_vdr"] = vdr["count_vdr"].replace('S', 0)
# vdr["pop"] = vdr["pop"].replace('S', 0)

#print(vdr[vdr.count_vdr == 'S'].shape[0]/vdr.shape[0])

# drop rows with suppressed values
#vdr = vdr.drop(vdr[vdr.count_vdr == 'S'].index)

# filter for valid counts
vdr = vdr[vdr.count_vdr != 'S']

# set types
#vdr = vdr.astype({"lzuid":np.int32, "mpoMaoriPacific":str, "ageband":str, "pop":np.int32, "count_vdr":np.int32})
vdr = vdr.astype({"mpoMaoriPacific":str, "ageband":str, "pop":np.int32, "count_vdr":np.int32})
vdr.head()

In [ ]:
# clinics
# these are currently manuallys set to the nearest data zone location (population centeroid)
clinics = pd.read_csv(os.path.join(data_dir, "akl_clinics.csv"), index_col="DZ2018")
clinics.head()

### Some basic plotting with pydeck

Can use this to select a location ID

In [ ]:
# deckgl show polygons and location ids
view_state = pydeck.ViewState(
    longitude=174.7633,
    latitude=-36.8485,
    zoom=11,    
    max_zoom=16,
    pitch=0,
    bearing=0
)

# default view
geojson = pydeck.Layer(
    "GeoJsonLayer",
    polys, # needs to be wgs84
    opacity=0.2,
    stroked=True,
    line_width_min_pixels=1,
    filled=True,           
    pickable=True,
    auto_highlight=True,
    get_fill_color=[128, 128, 128],
    get_line_color=[255, 255, 255],    
)

r = pydeck.Deck(
    layers=[geojson], 
    initial_view_state=view_state, 
    map_style='mapbox://styles/mapbox/light-v9',   
    tooltip = {
        "text": "Location: {id}"
    }
)
#r.to_html("geojson_layer.html", iframe_width="100%")
r.show()

### Journey times to a selected destination given a time budget

In [ ]:
print(clinics["name"])

In [ ]:
# get clinic location by name
clinic_name = "Rehab Plus"
clinic_loc = clinics.index[clinics["name"] == clinic_name].tolist()[0]
print(clinic_name, clinic_loc)

In [ ]:
loc = clinic_loc # or just set any location id
lon, lat = coords[loc]
destination = loc_idx[loc] # get odt index from location id
ot = odt[:, destination, :] # get origin-time matrix for this destination
print(ot.shape)

In [ ]:
# view ot matrix
fig, ax = plt.subplots(figsize=(15, 15))
ax.imshow(np.transpose(ot))
ax.set_xlabel("location index")
ax.set_ylabel("time index")

In [ ]:
# compute mean, std travel time
mean_tt = np.nanmean(ot, axis=1).reshape(-1, 1)
std_tt = np.nanstd(ot, axis=1).reshape(-1, 1)

In [ ]:
# create dataframe
ids = np.array(list(idx_loc.values())).reshape(-1, 1)
d = np.concatenate((ids, mean_tt, std_tt), axis=1)
df = pd.DataFrame(d, columns=["id", "mean_tt", "std_tt"])
df = df.astype({'id': 'int32'})
df = df.dropna()

# join with imd
df = df.join(imd, on="id")

In [ ]:
# threshold by mean_tt
threshold = 60 # minutes
df = df[df["mean_tt"] < threshold]

In [ ]:
# plot - note geojson data needs to be wgs84
fig = px.choropleth_mapbox(
        df,
        geojson=polys, 
        featureidkey="id",
        locations="id",        
        center = {"lat": lat, "lon": lon},
        mapbox_style="carto-positron",
        color="mean_tt",
        color_continuous_scale="Viridis",
        zoom=12)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

### Viable start locations for a trip to a selected destination given a time budget

* It doesn't seem like A->B->C with unconstrained A, C and time are possible with simple array operations. 
* A->B can be done this way, but for each journey AB(t) we must then check for a valid BC(t+delta_B) where delta_B is the time spent at B

##### Method

1. Start with AB (inbound from A to B) matrix, create travel time budget matrix B_AB with starting value t_budget for all valid journeys
2. Subtract travel times for each journey in AB from the budget matrix
3. Subtract delta_B, the time spent at B from the budget matrix and convert delta_B into a time index offset t_offset e.g. 60 minutes = 6 at 10 min resolution
4. Query the BC (outbound from B to C) matrix
5. For each journey in AB at time t with positive time remaining in B_AB:
  * calculate max_t_offset via remaining budget e.g. if remaining time for this journey is 60 minutes, max_t_offset is 6
  * create a slice matirx S_BC from the BC matrix at time t+t_offset:t+max_t_offset
  * create a new budget matrix B_S_BC to match the sliced BC matrix and set all values to the remaining time for this journey
  * subtract the travel times in S_BC from B_S_BC
  * store the resulting matrix
6. The resulting data structure is N x T x valid_T(j) x N, where N is the number of locations, T is the number of possible journey times and valid_T(j) is the number of valid journey times for each journey in AB

In [ ]:
def is_valid(a):        
    return np.logical_not(np.isnan(a))

def plot_time_budget(t_remain, t_max=60):
    plt.figure(figsize=(20, 10))
    plt.imshow(np.transpose(t_remain[:, :]), vmin=0, vmax=t_max)
    plt.title("time budget")
    plt.xlabel("location index")
    plt.ylabel("time index")
    plt.colorbar(orientation="horizontal")
    plt.show()   
    
def plot_valid(valid):
    plt.figure(figsize=(20, 5))
    plt.imshow(np.transpose(valid[:, :]))
    plt.title("valid journeys")
    plt.xlabel("location index")
    plt.ylabel("time index")    
    plt.show()      
    
def plot_journeys(journeys, t_max):
    plt.figure(figsize=(20, 10))
    plt.imshow(np.transpose(journeys[:, :]), vmax=t_max)
    plt.title("eta")
    plt.xlabel("location index")
    plt.ylabel("time index")
    plt.colorbar(orientation="horizontal")
    plt.show() 
    
def t_offset(t, t_delta):
    t_offset = int(t / t_delta)
    t_offset = t_offset + 1 if t % t_delta > 0 else t_offset
    return t_offset

def reduce(m):
    # given matirx of remaining times, reduce along time dimension to derive accessibility of each location 
    # as the ratio of valid to possible trips        
    
    nl, nt = m.shape        
    mask = m > 0    
    return mask.sum(axis=1) / nt    

In [ ]:
# A -> B

# inputs and constraints
dest_id = 7601026 # Auckland Uni Health Centre
lon, lat = coords[dest_id]
t_max = 2 * 60 # minutes available for itinerary
t_dest = 1 * 60 # minutes spent at destination
t_delta = 10 # step size (minutes) for the time dimension in the odt matrix

# select origin-time slice to the destination
dest_idx = loc_idx[dest_id] # get odt index from location id
AB = odt[:, dest_idx, :] # get origin-time matrix for the destination

In [ ]:
# identify valid journeys (invalid journeys have nan travel times)
valid = is_valid(AB)

# construct initial time budget matrix
t_remain = np.zeros_like(AB) # invalid journeys start with 0 time remaining
t_remain[valid] = t_max # valid journeys start with full time allowance

In [ ]:
# subtract travel time to destination
t_remain[valid] -= AB[valid]

# subtract time at destination
t_remain[valid] -= t_dest

# clip negative values to 0
t_remain[t_remain < 0] = 0 

# # plot remaining time
# plot_time_budget(t_remain, t_max)

# accessibility of B from A
acc_B = reduce(t_remain).reshape(-1, 1)

In [ ]:
# create dataframe
ids = np.array(list(idx_loc.values())).reshape(-1, 1)
d = np.concatenate((ids, acc_B), axis=1)
df = pd.DataFrame(d, columns=["id", "acc_B"])
df = df.astype({'id': 'int32'})
df = df.dropna()

In [ ]:
# plot - note geojson data needs to be wgs84
fig = px.choropleth_mapbox(
        df,
        geojson=polys, 
        featureidkey="id",
        locations="id",        
        center = {"lat": lat, "lon": lon},
        mapbox_style="carto-positron",
        color="acc_B",
        color_continuous_scale="bugn",
        zoom=12)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
# B->C

# calculate time offset for duration at B
t_dest_offset = t_offset(t_dest, t_delta)

# select BC matrix
BC = odt[dest_idx, :, :] # get matrix representing journeys starting at the target location

# create result matrix
nl, nt = AB.shape # location x time
AB_BC = np.zeros((nl, nt, nl))

_t_start = time.time()

# compute journey output matrices
# currently this is for all C, but in practice could be done for individual C based on user interaction

for jl in range(nl)[:]:
    for jt in range(nt)[:]:        
        # check for valid trips
        t_remain_j = t_remain[jl, jt]
        if  t_remain_j > 0:
            # calculate max t offset 
            t_max_offset = t_offset(t_remain_j, t_delta)
         
            # slice the BC matrix for valid journey times
            start_t = jt + t_dest_offset
            end_t = min(nt + 1, start_t + t_max_offset)
            BCj = BC[:, start_t:end_t]
            #print(t_remain_j, jt, start_t, end_t)
            
            if BCj.shape[1] == 0:
                continue
                                        
            # determine valid trip indices
            valid_BCj = is_valid(BCj)
            
            # create a budget matrix that matches the slice dimensions
            t_remain_BCj = np.zeros_like(BCj)
            # set valid trip values to the remaining time
            t_remain_BCj[valid_BCj] = t_remain_j
            
            # subtract the travel time from B to C
            t_remain_BCj[valid_BCj] -= BCj[valid_BCj]
            
            # clip negative values to 0
            t_remain_BCj[t_remain_BCj < 0] = 0            
            
            # reduce BCj matrix
            AB_BC[jl, jt] = reduce(t_remain_BCj)                        
                                
print(f"Time: {time.time() - _t_start:.2f} s")                

In [ ]:
# reduce the AB_BC matrix over time to get output location C accessibility
# from each input location A via time spent at B
acc_C = AB_BC.mean(axis=1)

In [ ]:
# visualise a specific C

C = 7600862
idx_C = loc_idx[C]
acc_C_idx = acc_C[idx_C].reshape(-1, 1)

# create dataframe
ids = np.array(list(idx_loc.values())).reshape(-1, 1)
d = np.concatenate((ids, acc_C_idx), axis=1)
df = pd.DataFrame(d, columns=["id", "acc_C"])
df = df.astype({'id': 'int32'})
df = df.dropna()

In [ ]:
# plot - note geojson data needs to be wgs84
fig = px.choropleth_mapbox(
        df,
        geojson=polys, 
        featureidkey="id",
        locations="id",        
        center = {"lat": lat, "lon": lon},
        mapbox_style="carto-positron",
        color="acc_C",
        color_continuous_scale="orrd",
        zoom=12)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

### Point to point travel times to investigate variablitily throughout the day

In [ ]:
from_loc = 7600316 # high stdev
to_loc = clinic_loc 

origin_idx = loc_idx[from_loc] # get odt index from location id
dest_idx = loc_idx[to_loc]

dt = odt[origin_idx, dest_idx, :] # get destination travel time series for this origin
dt = pd.Series(dt, index=pd.DatetimeIndex(list(t_idx.keys())))
print(dt.shape)

In [ ]:
dt.plot(figsize=(15, 5), xlabel="Departure Time", ylabel="ETA (minutes)")

In [ ]:
from_loc = 7600870 # low stdev
to_loc = clinic_loc 

origin_idx = loc_idx[from_loc] # get odt index from location id
dest_idx = loc_idx[to_loc]

dt = odt[origin_idx, dest_idx, :] # get destination travel time series for this origin
dt = pd.Series(dt, index=pd.DatetimeIndex(list(t_idx.keys())))
dt.plot(figsize=(15, 5), xlabel="Departure Time", ylabel="ETA (minutes)")

### Find the travel time from every location to every clinic

In [ ]:
# get clinic odt indexs
clinic_locs = clinics.index.tolist()
clinic_idxs = [loc_idx[l] for l in clinic_locs]

# get clinic odt
odt_clinic = odt[:, clinic_idxs, :]
print(odt_clinic.shape)

In [ ]:
# compute mean, std travel time
mean_tt = np.nanmean(odt_clinic, axis=-1)
std_tt = np.nanstd(odt_clinic, axis=-1)
print(mean_tt.shape)

In [ ]:
# find minimum mean time and its stdev
min_tt = np.zeros((mean_tt.shape[0], 1))
min_tt_std = np.zeros((mean_tt.shape[0], 1))

for i in range(mean_tt.shape[0]):
    min_t = mean_tt[i, 0]
    min_j = 0
    
    for j in range(1, mean_tt.shape[1]):
        if np.isnan(min_t) and not np.isnan(mean_tt[i, j]):
            min_t = mean_tt[i, j]
            min_j = j
        elif np.isnan(mean_tt[i, j]):
            pass
        elif mean_tt[i, j] < min_t:
            min_t = mean_tt[i, j]
            min_j = j
            
    min_tt[i] = min_t
    min_tt_std[i] = std_tt[i, min_j]
    #print(i, min_t, min_j, std_tt[i, min_j])        

In [ ]:
plt.figure(figsize=(10, 10))
plt.scatter(min_tt, min_tt_std)
plt.xlabel("Mean travel time to nearest clinic")
plt.ylabel("stdev travel time to nearest clinic")

In [ ]:
# use log travel travel times to plot on map more easily
log_min_tt = np.log10(min_tt)
log_min_tt_std = np.log10(min_tt_std)

# replace destinations with 0 travel time
log_min_tt[np.isneginf(log_min_tt)] = 0
log_min_tt_std[np.isneginf(log_min_tt_std)] = 0

In [ ]:
# create dataframe
ids = np.array(list(idx_loc.values())).reshape(-1, 1)
d = np.concatenate((ids, min_tt, min_tt_std, log_min_tt, log_min_tt_std), axis=1)
df = pd.DataFrame(d, columns=["id", "min_tt", "min_tt_std", "log_min_tt", "log_min_tt_std"])
df = df.astype({'id': 'int32'})
df = df.dropna()

# join with imd
df = df.join(imd, on="id")

In [ ]:
# plot - note geojson data needs to be wgs84
fig = px.choropleth_mapbox(
        df,
        geojson=polys, 
        featureidkey="id",
        locations="id",        
        center = {"lat": lat, "lon": lon},
        mapbox_style="carto-positron",
        color="log_min_tt",
        color_continuous_scale="Viridis",
        zoom=12)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
# plot - note geojson data needs to be wgs84
fig = px.choropleth_mapbox(
        df,
        geojson=polys, 
        featureidkey="id",
        locations="id",        
        center = {"lat": lat, "lon": lon},
        mapbox_style="carto-positron",
        color="log_min_tt_std",
        color_continuous_scale="Viridis",
        zoom=12)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
loc = 7601177
idx = loc_idx[loc]
min_tt_std[idx]

### Compared with deprivation

In [ ]:
df.columns

In [ ]:
# does accessibility to nearest clinic predict IMD Health Index?

x = "log_min_tt"
xlabel = "Travel time to nearest clinic"
y = "Health"
ylabel = f"{y} Index"

plt.figure(figsize=(10, 10))
plt.scatter(df[x], df[y])
plt.xlabel(xlabel)
plt.ylabel(ylabel)

In [ ]:
# loop through them all
Y = ['Census18_P', 'IMD18', 'Employment', 'Income', 'Crime', 'Housing', 'Health', 'Education', 'Access']
x = "log_min_tt"

for y in Y:        
    plt.figure(figsize=(10, 10))
    plt.scatter(df[x], df[y])
    plt.xlabel(x)
    plt.ylabel(y)        

In [ ]:
# rename id column and save
df.rename(columns={"id":"DZ2018"}).to_csv("imd-with-travel-time.csv", index=False)

In [ ]:
# join with vdr
df_vdr = df.join(vdr, on="id")
df_vdr.head()

In [ ]:
# select ethnicity and age combo
df_vdr[(df_vdr.mpoMaoriPacific == "MaoriPacific") & (df_vdr.ageband == "20-44")].head()

In [ ]:
x = "log_min_tt"

plt.figure(figsize=(10, 10))
for eth in ["MaoriPacific"]:
    for ageband in ["20-44", "45-64", "65+"]:
        sample = df_vdr[(df_vdr.mpoMaoriPacific == eth) & (df_vdr.ageband == ageband)].dropna()        

        plt.scatter(sample[x], 100 * sample["count_vdr"]/sample["pop"], label=f"{eth}, {ageband}", alpha=0.5)
        plt.xlabel("travel time (log10 minutes) to nearest clinic")
        plt.ylabel("vdr as % of pop")  
        plt.ylim(0, 100)
        
plt.title(f"{eth} VDR vs travel time")
plt.legend()

In [ ]:
x = "log_min_tt"

plt.figure(figsize=(10, 10))
for eth in ["nMnP"]:
    for ageband in ["20-44", "45-64", "65+"]:
        sample = df_vdr[(df_vdr.mpoMaoriPacific == eth) & (df_vdr.ageband == ageband)].dropna()        

        plt.scatter(sample[x], 100 * sample["count_vdr"]/sample["pop"], label=f"{eth}, {ageband}", alpha=0.5)
        plt.xlabel("travel time (log10 minutes) to nearest clinic")
        plt.ylabel("vdr as % of pop")  
        plt.ylim(0, 100)
        
plt.title(f"{eth} VDR vs travel time")
plt.legend()

In [ ]:
# maps

eth = "MaoriPacific"
#eth = "nMnP"
ageband = "45-64"
print(f"VDR as % of pop. for {eth}, aged {ageband}")
      
sample = df_vdr[(df_vdr.mpoMaoriPacific == eth) & (df_vdr.ageband == ageband)].dropna()
sample["vdr_perc"] = 100 * sample["count_vdr"] / sample["pop"]

# plot - note geojson data needs to be wgs84
fig = px.choropleth_mapbox(
        sample,
        geojson=polys, 
        featureidkey="id",
        locations="id",        
        center = {"lat": lat, "lon": lon},
        mapbox_style="carto-positron",
        color="vdr_perc",
        color_continuous_scale="Viridis",
        zoom=12)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
np.unique(df.id.values).shape[0]

In [ ]:
np.unique(df_vdr.dropna().id.values).shape[0]